In [1]:
import csv
import os
import pymc as pm
from pymc import do, observe
import pandas as pd
import numpy as np
import arviz as az
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from pytensor import tensor as pt
import pickle as pkl
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import itertools as it
import country_converter as cc
import math
from sklearn.linear_model import LinearRegression

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Convert QGIS temperature output files to single unweighted temperature file

In [ ]:
curr_month = 1
curr_year = 1900
dir = "../data/processed/temp_by_country_raster_band/"
monthly_temp_data_by_country = {"Country":[],"Year":[],"Month":[],"Mean_Temp":[]}
for file in sorted(os.listdir(dir)):
    if "country_temp_raster_band_" in file:
        month_file = pd.read_csv(dir + file)
        country_codes = month_file["GMI_CNTRY"]
        mean_temps = month_file["_mean"]
        for index in range(len(country_codes)):
            country = country_codes[index]
            temp = mean_temps[index]
            monthly_temp_data_by_country["Country"].append(country)
            monthly_temp_data_by_country["Year"].append(curr_year)
            monthly_temp_data_by_country["Month"].append(curr_month)
            monthly_temp_data_by_country["Mean_Temp"].append(temp)
        curr_month += 1
        if curr_month == 13:
            curr_month = 1
            curr_year += 1
zipped = zip(monthly_temp_data_by_country["Country"],monthly_temp_data_by_country["Year"],monthly_temp_data_by_country["Month"],monthly_temp_data_by_country["Mean_Temp"])
zipped = list(sorted(zipped))
monthly_temp_data_by_country["Country"] = [row[0] for row in zipped]
monthly_temp_data_by_country["Year"] = [row[1] for row in zipped]
monthly_temp_data_by_country["Month"] = [row[2] for row in zipped]
monthly_temp_data_by_country["Mean_Temp"] = [row[3] for row in zipped]
pd.DataFrame.from_dict(monthly_temp_data_by_country).to_csv("../data/processed/unweighted_monthly_temp_by_country.csv")          

# create formatted dataset from pop-weighted country temp data by month

In [25]:
result_dict = {"Country":[],"Year":[],"Avg_PopWeighted_Temp":[]}
# data = pd.read_csv("../data/burke/data/input/nc/pop_weighted_country_temps_by_month.csv")
data = pd.read_csv("../data/burke/data/input/nc/unweighted_country_temps_by_month.csv")
col_prefix = "unweighted_monthly_temp.mean.X"
years = [str(year) for year in list(range(1900,2018))]
months = [str(month) if month >= 10 else "0"+str(month) for month in list(range(1,13))]
for _, row in data.iterrows():
    country = row["country"]
    for year in years:
        all_vals_by_year = []
        for month in months:
            col_name = col_prefix + year + "." + month + ".01"
            all_vals_by_year.append(row[col_name])
        result_dict["Country"].append(country)
        result_dict["Year"].append(year)
        result_dict["Mean_Temp"].append(np.nanmean(all_vals_by_year))
pd.DataFrame.from_dict(result_dict).to_csv("../data/burke/data/input/custom_monthly_unweighted_temp_by_country.csv")

/var/folders/q3/myxq41dd4_9c_rx76zlk8f3r0000gn/T/ipykernel_41576/2567336226.py:17: RuntimeWarning: Mean of empty slice
  result_dict["Avg_PopWeighted_Temp"].append(np.nanmean(all_vals_by_year))


# create formatted dataset from unweighted country precip data by month

In [17]:
result_dict = {"Country":[],"Year":[],"Unweighted_Precipitation":[]}
# data = pd.read_csv("../data/burke/data/input/nc/pop_weighted_country_temps_by_month.csv")
data = pd.read_csv("../data/burke/data/input/unweighted_country_precip_by_month.csv")
col_prefix = "unweighted_monthly_precip.mean.precip_clipped_by_country_mask_"
years = [str(year) for year in list(range(1900,2015))]
months = [str(month) if month >= 10 else "0"+str(month) for month in list(range(1,13))]
for _, row in data.iterrows():
    month_count = 1
    country = row["country"]
    for year in years:
        all_vals_by_year = []
        for month in months:
            col_name = col_prefix + str(month_count)
            all_vals_by_year.append(row[col_name])
            month_count += 1
        result_dict["Country"].append(country)
        result_dict["Year"].append(year)
        result_dict["Unweighted_Precipitation"].append(np.nanmean(all_vals_by_year))
pd.DataFrame.from_dict(result_dict).to_csv("../data/burke/data/input/custom_monthly_unweighted_precip_by_country.csv")

/tmp/ipykernel_80090/1184637301.py:20: RuntimeWarning: Mean of empty slice
  result_dict["Unweighted_Precipitation"].append(np.nanmean(all_vals_by_year))


[4.1454815864563, 4.1969313621521, 10.0919666290283, 3.15712332725525, 2.14509987831116, 8.84110260009766, 20.4474258422852, 8.90329170227051, 7.58338975906372, 15.6217288970947, 6.25479507446289, 4.02499961853027]


In [28]:
# burke dataset missing data
burke_data = pd.read_csv("../data/burke/data/input/GrowthClimateDataset.csv")
temp = burke_data.UDel_temp_popweight
precp = burke_data.UDel_precip_popweight
gdp = burke_data.growthWDI
countries = burke_data.iso
years = burke_data.year

gdp_no_temp = 0
temp_no_gdp = 0
no_both = 0

for i in range(len(temp)):
    if years[i] < 2011 and np.isnan(temp[i]) and np.isnan(gdp[i]):
        no_both += 1
    elif years[i] < 2011 and np.isnan(temp[i]) and not np.isnan(gdp[i]):
        gdp_no_temp += 1
    elif years[i] < 2011 and not np.isnan(temp[i]) and np.isnan(gdp[i]):
        temp_no_gdp += 1

print(gdp_no_temp)
print(temp_no_gdp)
print(no_both)

925
767
266


# proof-of-concept for country-specific temp priors

In [62]:
data = pd.read_csv("../data/burke/data/input/GrowthClimateDatasetTruncated.csv")

indices_to_drop = []
no_nan_cols = ["UDel_temp_popweight","UDel_precip_popweight","growthWDI"]
for index, row in enumerate(data.itertuples()):
    if any(np.isnan(getattr(row,col)) for col in no_nan_cols):
        indices_to_drop.append(index)
data_no_missing = data.drop(indices_to_drop)

precip_scaler, gdp_scaler, temp_scaler = StandardScaler(), StandardScaler(), StandardScaler()
precip_scaled = precip_scaler.fit_transform(np.array(data_no_missing.UDel_precip_popweight).reshape(-1,1)).flatten()
gdp_scaled = gdp_scaler.fit_transform(np.array(data_no_missing.growthWDI).reshape(-1,1)).flatten()
temp_scaled = temp_scaler.fit_transform(np.array(data_no_missing.UDel_temp_popweight).reshape(-1,1)).flatten()

data_len = len(data_no_missing.year)
year_mult_mat = [np.zeros(data_len) for year in set(data_no_missing.year)]
country_mult_mat = [np.zeros(data_len) for country in set(data_no_missing.iso)]

country_index = -1
curr_country = ""
for row_index, row in enumerate(data_no_missing.itertuples()):
    if row.iso != curr_country:
        country_index += 1
        curr_country = row.iso
    year_index = row.year - 1961
    country_mult_mat[country_index][row_index] = 1
    year_mult_mat[year_index][row_index] = 1

grad_effects_data = np.transpose(np.array(data_no_missing.loc[:, data.columns.str.startswith(('_y'))]))

In [73]:
data = pd.read_csv("../data/burke/data/input/GrowthClimateDatasetTruncated.csv")
precip_scaler, gdp_scaler, temp_scaler = StandardScaler(), StandardScaler(), StandardScaler()
precip_scaled = precip_scaler.fit_transform(np.array(data.UDel_precip_popweight).reshape(-1,1)).flatten()
gdp_scaled = gdp_scaler.fit_transform(np.array(data.growthWDI).reshape(-1,1)).flatten()
temp_scaled = temp_scaler.fit_transform(np.array(data.UDel_temp_popweight).reshape(-1,1)).flatten()

data_len = len(data.year)
year_mult_mat = [np.zeros(data_len) for year in set(data.year)]
country_mult_mat = [np.zeros(data_len) for country in set(data.iso)]

country_index = -1
curr_country = ""
for row_index, row in enumerate(data.itertuples()):
    if row.iso != curr_country:
        country_index += 1
        curr_country = row.iso
    year_index = row.year - 1960
    country_mult_mat[country_index][row_index] = 1
    year_mult_mat[year_index][row_index] = 1

grad_effects_data = np.transpose(np.array(data.loc[:, data.columns.str.startswith(('_y'))]))

In [82]:
with pm.Model() as model:

    country_coefs_temp_prior = pt.expand_dims(pm.Normal("country_coefs_temp_prior", 0, 1, shape=(len(set(data.iso)))),axis=1)
    temp_prior = pm.Deterministic("temp_prior",pt.sum(country_coefs_temp_prior*country_mult_mat,axis=0))
    # temp_prior = pm.Normal("temp_prior", 0, 1)
    temp_std = pm.HalfNormal("temp_std", 10)
    temp_posterior = pm.Normal("temp_posterior", temp_prior, temp_std, observed=temp_scaled)

    country_coefs_precip_prior = pt.expand_dims(pm.Normal("country_coefs_precip_prior", 0, 1, shape=(len(set(data.iso)))),axis=1)
    precip_prior = pm.Deterministic("precip_prior",pt.sum(country_coefs_precip_prior*country_mult_mat,axis=0))
    # precip_prior = pm.Normal("precip_prior", 0, 1)
    precip_std = pm.HalfNormal("precip_std", 10)
    precip_posterior = pm.Normal("precip_posterior", precip_prior, precip_std, observed=precip_scaled)

    gdp_intercept = pm.Normal('gdp_intercept',0,10)
    temp_gdp_coef = pm.Normal('temp_gdp_coef',0,10)
    temp_sq_gdp_coef = pm.Normal('temp_sq_gdp_coef',0,10)
    precip_gdp_coef = pm.Normal("precip_gdp_coef",0,10)
    precip_sq_gdp_coef = pm.Normal("precip_sq_gdp_coef",0,10)

    # year_coefs = pt.expand_dims(pm.Normal("year_coefs", 0, 5, shape=(len(set(data_no_missing.year)))),axis=1)
    year_coefs = pt.expand_dims(pm.Normal("year_coefs", 0, 5, shape=(len(set(data.year)))),axis=1)
    year_fixed_effects = pm.Deterministic("year_fixed_effects",pt.sum(year_coefs*year_mult_mat,axis=0))

    # country_coefs = pt.expand_dims(pm.Normal("country_coefs", 0, 5, shape=(len(set(data_no_missing.iso)))),axis=1)
    country_coefs = pt.expand_dims(pm.Normal("country_coefs", 0, 5, shape=(len(set(data.iso)))),axis=1)
    country_fixed_effects = pm.Deterministic("country_fixed_effects",pt.sum(country_coefs*country_mult_mat,axis=0))

    gradual_effect_coefs = pt.expand_dims(pm.Normal("grad_effect_coefs", 0, 5, shape=(len(grad_effects_data))),axis=1)
    gradual_effects = pm.Deterministic("grad_effects",pt.sum(gradual_effect_coefs*grad_effects_data,axis=0))
    
    gdp_prior = pm.Deterministic(
        "gdp_prior",
        gdp_intercept +
        (temp_posterior * temp_gdp_coef) +
        (temp_sq_gdp_coef * pt.sqr(temp_posterior)) +
        (precip_posterior * precip_gdp_coef) +
        (precip_sq_gdp_coef * pt.sqr(precip_posterior)) +
        year_fixed_effects +
        country_fixed_effects +
        gradual_effects
    )
    
    gdp_std = pm.HalfNormal('gdp_std', sigma=10)
    gdp_posterior = pm.Normal('gdp_posterior', mu=gdp_prior, sigma=gdp_std, observed=gdp_scaled)

    prior = pm.sample_prior_predictive()
    trace = pm.sample()
    posterior = pm.sample_posterior_predictive(trace, extend_inferencedata=True)

/home/hayden_freedman/pymc_dev.venv/lib/python3.10/site-packages/pymc/model/core.py:1323: ImputationWarning: Data in temp_posterior contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/home/hayden_freedman/pymc_dev.venv/lib/python3.10/site-packages/pymc/model/core.py:1323: ImputationWarning: Data in precip_posterior contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/home/hayden_freedman/pymc_dev.venv/lib/python3.10/site-packages/pymc/model/core.py:1323: ImputationWarning: Data in gdp_posterior contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Sampling: [country_coefs, country_coefs_precip_prior, country_coefs_temp_prior, gdp_intercept, gdp_posterior_observed, gdp_posterior_unobserved, gdp_std, grad_effect_coefs, precip_

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1075 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Sampling: [gdp_posterior_observed, gdp_posterior_unobserved, precip_posterior_observed, temp_posterior_observed]


In [65]:
trace_1 = trace
posterior_1 = posterior

In [66]:
print(np.mean(posterior_1.posterior.temp_gdp_coef.data))
print(np.mean(posterior_1.posterior.temp_sq_gdp_coef.data))
print(np.mean(posterior_1.posterior.precip_gdp_coef.data))
print(np.mean(posterior_1.posterior.precip_sq_gdp_coef.data))

1.5022163626463039
-0.797895699922554
0.18873404810271932
0.13316074718098558


In [75]:
trace_2 = trace
posterior_2 = posterior

In [76]:
print(np.mean(posterior_2.posterior.temp_gdp_coef.data))
print(np.mean(posterior_2.posterior.temp_sq_gdp_coef.data))
print(np.mean(posterior_2.posterior.precip_gdp_coef.data))
print(np.mean(posterior_2.posterior.precip_sq_gdp_coef.data))

-0.11842936695809973
-0.6886470889358016
0.002734180937188714
0.06128721765641807


In [81]:
print(posterior_2.posterior.temp_posterior_unobserved.data[0][0])
print(np.mean(posterior_2.posterior.temp_posterior_unobserved.data))
print(np.std(posterior_2.posterior.temp_posterior_unobserved.data))

[-2.57238346 -0.24040984 -0.71098405  1.06328328 -1.28347795 -1.87846285
 -1.26125807 -0.80732258  0.12412204  0.91232099  0.25623295 -0.23307003
 -1.93645254 -1.13541942 -0.68624079]
-0.023601708127025417
0.8612753605441308


In [83]:
trace_3 = trace
posterior_3 = posterior

In [84]:
print(np.mean(posterior_3.posterior.temp_gdp_coef.data))
print(np.mean(posterior_3.posterior.temp_sq_gdp_coef.data))
print(np.mean(posterior_3.posterior.precip_gdp_coef.data))
print(np.mean(posterior_3.posterior.precip_sq_gdp_coef.data))

1.0696387494285733
-0.8339804300002953
0.18833593015714178
0.12195729156337243


In [89]:
print(posterior_3.posterior.temp_posterior_unobserved.data[0][0])
print(np.mean(posterior_3.posterior.temp_posterior_unobserved.data))
print(np.std(posterior_3.posterior.temp_posterior_unobserved.data))

[-0.92223356 -0.81234973 -0.798446    0.81519524  0.55951362  0.71368768
 -0.88402724 -0.7376646  -0.73193389  1.55575454  1.37519756 -0.12757303
 -1.69273233 -1.69232622 -1.72259294]
-0.3432309958909134
1.0696323687060416
